In [1]:
import flwr as fl

In [ ]:
import tensorflow as tf
# from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tinydb import TinyDB, Query

db = TinyDB('../Application/dataset.csv')

(x_train, y_train), (x_test, y_test) = db.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


In [2]:
fl.server.start_server(config=fl.server.ServerConfig(num_rounds=3))

INFO flwr 2023-07-06 03:09:25,482 | app.py:148 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2023-07-06 03:09:25,522 | app.py:168 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2023-07-06 03:09:25,523 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-06 03:09:25,524 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-06 03:09:53,642 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-06 03:09:53,643 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-07-06 03:09:53,644 | server.py:101 | FL starting
DEBUG flwr 2023-07-06 03:10:25,695 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-07-06 03:10:34,373 | server.py:232 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-07-06 03:10:34,375 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-07-06 03:10:34,

History (loss, distributed):
	round 1: 0.5859200954437256
	round 2: 0.5260496139526367
	round 3: 0.45686882734298706